<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

- Unslothを使っています。Unslothについては、[こちらのガイド](https://docs.unsloth.ai/get-started/install-and-update)をご参照ください。
- このノートブックは、 [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme)ライセンスが適用されます。
- このノートブックのオリジナルは、[こちらのノートブック](https://github.com/unslothai/notebooks/blob/main/nb/Qwen3_(4B)-Instruct.ipynb)です。
- このノートブックは、Qwen3 0.6Bのモデルに対して、Google DriveにアップロードされたCSV形式（文字コードはBOM付きのUTF-8）の日本語データセットを使ってファインチューニングができるように改造しました。
- このノートブックは、Google Colab無料プランで使用可能なNVIDIA Tesla T4 GPUで動かすことができます。


### Step0.ファインチューニング向けデータセットの用意

■ 今回使用するデータセット「faq-dataset.csv」は、次のような構造のCSVファイルで、文字コードはUTF-8(BOM付き)です。列は、`conversations`のみです。文字コードがUTF-8(BOM付き)であることで、Microsoft Excelで開いた際に文字化けしません。

| conversations |
| :---- |
| "[[{""role"":""user"",""content"":""キルチェーン（Cyber Kill Chain）とは何ですか？""},{""role"":""assistant"",""content"":""キルチェーンとは、攻撃の一連の段階（偵察・侵入・実行など）を整理したモデルです。各段階で対策を講じることで攻撃を阻止できます。""}]]" |
| "[[{""role"":""user"",""content"":""メールセキュリティでSPFとは何ですか？""},{""role"":""assistant"",""content"":""SPFは送信元メールサーバの正当性を検証する仕組みで、なりすましメール対策に利用されます。""}]]" |

■ ShareGPT形式で作成してください。

■ 120行のデータセットで動作確認してあります。ファインチューニング向けなので、最低でも100行は超えるように自作しましょう。動作確認結果と同じく、120行にしても良いですし、がんばって200行を超えても構いません。

■ CSVファイルでは、カンマで区切られたデータの横一列を"行"、縦一列を"列"と呼びます。

■ 作成したデータセットは、Google Driveにアップロードしましょう。Google Driveのアカウントが無い場合は、Googleアカウントを[新規取得](https://www.google.com/intl/ja/account/about/)してください。

### Step1.Unslothのインストール
トラブルを減らすために動作確認ができたソフトウェアのバージョンに固定しています。

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps {xformers}
    !pip install --no-deps bitsandbytes==0.49.0
    !pip install --no-deps accelerate==1.12.0
    !pip install --no-deps peft==0.18.0
    !pip install --no-deps trl==0.26.2
    !pip install --no-deps triton==3.5.1
    !pip install --no-deps cut_cross_entropy==25.1.1
    !pip install --no-deps unsloth_zoo==2026.1.2
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth==2025.12.10
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Step2.Google Driveのマウント

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step3.ファインチューニング対象のモデルの指定

In [3]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B",
    max_seq_length = 8192, # 1024 , 2048 , 4096 , 8192 : コンテキスト長の指定。8192以上もできます。
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.10: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

LoRA アダプターを追加

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.12.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Step4.データの準備と読み込み
会話形式のファインチューンにはQwen3形式を採用しています。データセットには、ShareGPT形式で使用しています。Qwen3は、以下のような会話を表現します。

```
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>

```

チャットテンプレートに、`qwen3-instruct`を使用します。

In [5]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-instruct",
)

Google Drive上のデータセット読み込み

In [7]:
from datasets import load_dataset

# CSVの読み込み
sheet_path = '/content/drive/MyDrive/conversations-2nd.csv'
dataset = load_dataset(
    "csv",
    data_files={"train": sheet_path},
    column_names=["conversations"],
    skiprows=1,
    encoding="utf-8-sig"
)["train"]

Generating train split: 0 examples [00:00, ? examples/s]

読み込んだデータセットの型を確認。

In [8]:
print("A:", type(dataset[0]["conversations"]))

A: <class 'str'>


読み込んだデータセットを、Pythonで扱えるように、list型に変更する。

In [9]:
import json

dataset = dataset.map(lambda x: {"conversations": json.loads(x["conversations"])[0]})

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

ファインチューニングのために、「standardize_data_formats」を使用してデータセットを、必要な形式に変換します。

In [10]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/120 [00:00<?, ? examples/s]

データセットの１つ目を確認します。

In [11]:
dataset[0]

{'conversations': [{'content': 'サイバーセキュリティにおけるCIAトライアドとは何ですか？',
   'role': 'user'},
  {'content': 'CIAトライアドとは、情報セキュリティの基本原則である「機密性（Confidentiality）」「完全性（Integrity）」「可用性（Availability）」の3要素を指します。これらをバランスよく維持することが、安全な情報システムの設計・運用において重要です。',
   'role': 'assistant'}]}

100 行目がどのようになっているか見てみましょう。

ここで、`Qwen-3` のチャット テンプレートを会話に適用し、`text`に保存する必要があります。

In [12]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

チャットテンプレートにより、100行目のデータがどのようになったか見ましょう。


In [13]:
dataset[100]['text']

'<|im_start|>user\n権限昇格攻撃とはどのような攻撃ですか？<|im_end|>\n<|im_start|>assistant\n権限昇格攻撃とは、低い権限のアカウントから管理者権限などを不正に取得する攻撃です。システム全体の侵害につながります。<|im_end|>\n'

<a name="Train"></a>
### Step5.モデルをトレーニング(学習)しましょう。
高速化のため60ステップで実行しますが、`num_train_epochs=1` に設定してフル実行し、`max_steps=None` をオフにすることもできます

In [14]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/120 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


また、Unslothの`train_on_completions`メソッドを使用して、アシスタントの出力のみで学習し、ユーザーの入力による損失を無視します。これにより、ファインチューンの精度が向上します。

In [15]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=6):   0%|          | 0/120 [00:00<?, ? examples/s]

もう一度100行目を出力してみましょう。

In [16]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|im_start|>user\n権限昇格攻撃とはどのような攻撃ですか？<|im_end|>\n<|im_start|>assistant\n権限昇格攻撃とは、低い権限のアカウントから管理者権限などを不正に取得する攻撃です。システム全体の侵害につながります。<|im_end|>\n'

次に、マスクされた例を出力してみましょう。答えだけが表示されるはずです。

In [17]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                       権限昇格攻撃とは、低い権限のアカウントから管理者権限などを不正に取得する攻撃です。システム全体の侵害につながります。<|im_end|>\n'

In [18]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
0.709 GB of memory reserved.


モデルのトレーニングを実行しましょう。

In [19]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 120 | Num Epochs = 4 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,185,088 of 616,235,008 (3.28% trained)


Step,Training Loss
1,3.066400
2,3.062000
3,2.834500
4,2.607900
5,2.455500
6,2.517700
7,2.049200
8,2.233100
9,2.205900
10,2.255300


In [20]:
# @title 使用したメモリ量（VRAM）
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

171.3574 seconds used for training.
2.86 minutes used for training.
Peak reserved memory = 1.053 GB.
Peak reserved memory for training = 0.344 GB.
Peak reserved memory % of max memory = 7.143 %.
Peak reserved memory for training % of max memory = 2.334 %.


<a name="Inference"></a>
### Step6.トレーニング後の動作確認
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for instruct inference are `temperature = 0.7, top_p = 0.8, top_k = 20`

For reasoning chat based inference, `temperature = 0.6, top_p = 0.95, top_k = 20`

In [21]:
messages = [
    {"role" : "user", "content" : "セキュリティポリシーとは？"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1000, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

セキュリティポリシーとは、組織内で共有されている安全な仕組みやルールの総合で、組織の内部安全性を維持するための基盤となる文書やマニュアルです。<|im_end|>


<a name="Save"></a>
### Step7.モデルの保存
モデルを LoRA アダプターとして保存するには、ローカル保存の場合は `save_pretrained`を使用します。

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [22]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

ここで、推論用に保存した LoRA アダプターをロードする場合は、`False` を `True` に設定します。

In [23]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 8192,
        load_in_4bit = True,
    )

#### GGUF / llama.cpp Conversion
`GGUF` / `llama.cpp` への保存をネイティブでサポートしました。`llama.cpp` をクローンし、デフォルトで `q8_0` に保存します。`q4_k_m` などのすべての手法が利用可能です。ローカル保存には `save_pretrained_gguf` を、HF へのアップロードには `push_to_hub_gguf` を使用してください。

サポートされている量子化手法（[Wiki ページ](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options) の全リストをご覧ください）：
* `q8_0` - 高速変換。リソース消費量は多いですが、一般的には許容範囲です。
* `q4_k_m` - 推奨。attention.wv と feed_forward.w2 テンソルの半分に Q6_K を使用し、それ以外は Q4_K を使用します。
* `q5_k_m` - 推奨。 attention.wv と feed_forward.w2 のテンソルの半分には Q6_K を使用し、それ以外は Q5_K を使用します。

GGUF に Hugging Face アカウントをプッシュする場合は、if False を if True に設定し、Hugging Face トークンとアップロード場所を追加してください。デフォルトでは、8ビット量子化で、GGUF形式をexport_modelフォルダに保存します。

In [24]:
# Save to 8bit Q8_0
if True:
    model.save_pretrained_gguf("export_model", tokenizer, quantization_method = "q8_0")
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("export_model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("export_model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:11<00:00, 11.49s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:11<00:00, 11.28s/it]


Unsloth: Merge process complete. Saved to `/content/export_model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['qwen3-0.6b.F16.gguf']
Unsloth: [2] Conve

### Step8.保存したモデルを使い、Ollamaで推論
Modelfileと、.ggufで終わるGGUFファイル（＝モデル本体）があることを確認すること。Modelfileがない場合は、生成処理に時間がかかっているだけなので、少し待つ。

#### Ollamaのインストール

In [25]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


#### Ollamaの起動

In [26]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3) # Ollamaがロードされるまで数秒間待ちます。

#### モデルファイルの確認

In [27]:
print(tokenizer._ollama_modelfile)


FROM {__FILE_LOCATION__}
TEMPLATE """{{- if .Messages }}
{{- if or .System .Tools }}<|im_start|>system
{{- if .System }}
{{ .System }}
{{- end }}
{{- if .Tools }}

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{{- range .Tools }}
{"type": "function", "function": {{ .Function }}}
{{- end }}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>
{{- end }}<|im_end|>
{{ end }}
{{- range $i, $_ := .Messages }}
{{- $last := eq (len (slice $.Messages $i)) 1 -}}
{{- if eq .Role "user" }}<|im_start|>user
{{ .Content }}<|im_end|>
{{ else if eq .Role "assistant" }}<|im_start|>assistant
{{ if .Content }}{{ .Content }}
{{- else if .ToolCalls }}<tool_call>
{{ range .ToolCalls }}{"name": "{{ .Function.Name }}", "arguments": {{ .Func

#### Ollamaと互換性のあるモデルを作成

In [28]:
!ollama create unsloth_qwen3_0.6B_FT -f ./Modelfile

#### Ollamaで推論を実行

In [29]:
!curl http://localhost:11434/api/chat -d '{ \
"model": "unsloth_qwen3_0.6B_FT",\
"messages": [{ "role": "user", "content": "情報セキュリティポリシーについて解説して。" }],\
"stream": false\
}'

{"model":"unsloth_qwen3_0.6B_FT","created_at":"2026-01-07T01:59:29.817307624Z","message":{"role":"assistant","content":"情報セキュリティポリシーとは、組織全体で情報セキュリティを守るための基本的なルールやガイドラインを定めた文書です。"},"done":true,"done_reason":"stop","total_duration":2632995445,"load_duration":2210897819,"prompt_eval_count":22,"prompt_eval_duration":27686642,"eval_count":38,"eval_duration":319875750}

### Step9.ファインチューニングしたモデルをローカルで使用する
通信状況により、1時間以上かかることがあります。

#### モデルのダウンロード
ファインチューニングしたモデルをダウンロードします。ダウンロードしたモデルは、LM Studioで読み込み、文章生成に使うことができます。PCやサーバー上のOllamaで使いたい場合は、ファインチューニングしたモデルのダウンロード後、先ほど作成したModelfileもダウンロードしましょう。

In [ ]:
from google.colab import files
files.download('/content/qwen3-0.6b.Q8_0.gguf')

モデルファイルのダウンロード（Ollamaでは必須）

In [ ]:
from google.colab import files
files.download('/content/Modelfile')

#### LM Studio のインストールとモデルの動作確認

1. LM Stduioの[ホームページ](https://lmstudio.ai/)から、「Download」ボタンをクリックします。インストーラー（Windows環境の場合は、2026年1月現在 LM-Studio-0.3.36-1-x64.exe）をダウンロードします。
2. インストーラーを起動し、LM Studioをインストールします。
3. LM Stduioを起動します。
4. ターミナルアプリを起動します。WindowsやMacならば、「ターミナル」のこと。
5. ターミナルアプリで、モデルをダウンロードしたフォルダ（ディレクトリ）に移動します。
6. lms import qwen3-0.6b.Q8_0.gguf を実行します。
7. LM Studioにダウンロードしたモデル「qwen3-0.6b.Q8_0.gguf」が取り込まれます。
8. LM Studioでチャット機能を使い、動作確認します。

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
